# Lab 5: Direct Lake Framing

## Overview

This lab explores framing, the process by which a Direct Lake model tracks and synchronises with changes in the underlying lakehouse tables. You will observe automatic framing in action, disable it to perform batch data modifications, and then manually reframe the model to pick up accumulated changes.

### Workshop Flow

1. Connect to the AdventureWorks lakehouse and semantic model
2. Create a simple Power BI report to observe data changes
3. Review Delta table history for the fact and dimension tables
4. Perform insert, update, and delete operations with automatic framing enabled
5. Disable automatic framing and repeat the data modifications
6. Manually reframe the model to synchronise the accumulated changes

### Key Concepts

- **Automatic framing** keeps the model in sync with data changes in near real-time
- **Manual framing** gives you control over when the model picks up changes, which is useful during large batch operations
- **Delta history** records every data modification, providing an audit trail for framing decisions
- **Reframing** is the explicit operation that synchronises the model with the current state of the Delta tables

### Learning Objectives

- Understand the difference between automatic and manual framing
- Observe how insert, update, and delete operations trigger framing events
- Learn when to disable automatic framing for performance during batch processing
- Perform a manual reframe operation

**Prerequisites:** Lab 4 completed

---

*Deutsche Version:*

# Lab 5: Direct Lake Framing

## Uebersicht

Dieses Lab untersucht Framing, den Prozess, durch den ein Direct Lake-Modell Aenderungen in den zugrunde liegenden Lakehouse-Tabellen verfolgt und synchronisiert. Sie beobachten automatisches Framing in Aktion, deaktivieren es fuer Batch-Datenaenderungen und fuehren dann manuelles Reframing durch, um angesammelte Aenderungen zu uebernehmen.

### Wichtige Konzepte

- **Automatisches Framing** haelt das Modell nahezu in Echtzeit mit Datenaenderungen synchron
- **Manuelles Framing** gibt Ihnen Kontrolle darueber, wann das Modell Aenderungen uebernimmt
- **Delta-Historie** zeichnet jede Datenaenderung auf
- **Reframing** ist die explizite Operation, die das Modell mit dem aktuellen Zustand der Delta-Tabellen synchronisiert

**Voraussetzungen:** Lab 4 abgeschlossen

## Step 1: Install Required Libraries

Install the Semantic Link Labs library for Direct Lake framing operations.

---

*Installieren Sie die Semantic Link Labs-Bibliothek fuer Direct Lake Framing-Operationen.*

In [ ]:
%pip install -q semantic-link-labs

## Step 2: Import Libraries and Set Variables

Import the required Python libraries and set up lakehouse and semantic model variables.

---

*Importieren Sie die erforderlichen Python-Bibliotheken und richten Sie Lakehouse- und Semantic Model-Variablen ein.*

In [ ]:
# Import libraries for Direct Lake operations and data manipulation
import sempy_labs as labs
from sempy_labs import report
import time
import pandas

# Configure lakehouse and semantic model names
LakehouseName = "AdventureWorks"
lakehouses = labs.list_lakehouses()["Lakehouse Name"]
for l in lakehouses:
    if l.startswith("Adventure"):
        LakehouseName = l

# Set up workspace and lakehouse identifiers
SemanticModelName = f"{LakehouseName}_model"
workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]

## Step 3: Connect to Lakehouse and Validate Setup

Verify the lakehouse and semantic model are accessible and ready for framing operations.

---

*Ueberpruefen Sie, ob Lakehouse und Semantic Model zugaenglich und bereit fuer Framing-Operationen sind.*

In [ ]:
# Validate workspace connection and display setup information
print(f"Workspace ID: {workspaceId}")
print(f"Lakehouse Name: {LakehouseName}")
print(f"Lakehouse ID: {lakehouseId}")
print(f"Semantic Model Name: {SemanticModelName}")

# Verify lakehouse exists and is accessible
try:
    lakehouse_info = labs.list_lakehouses()
    print(f"\nAvailable Lakehouses: {len(lakehouse_info)} found")
    print("Setup validation complete!")
except Exception as e:
    print(f"Error accessing lakehouse: {e}")

## Step 4: Load Report Generation Libraries

Import helper functions for creating a Power BI report that will visualise data changes during the framing exercises.

---

*Importieren Sie Hilfsfunktionen zum Erstellen eines Power BI-Berichts, der Datenaenderungen waehrend der Framing-Uebungen visualisiert.*

In [ ]:
# Import specialized helper functions for Power BI operations and framing analysis
from sempy_labs._helper_functions import (
    resolve_report_id,
    format_dax_object_name,
    resolve_dataset_from_report,
    _conv_b64,
    _extract_json,
    _add_part,
    _decode_b64,
    resolve_workspace_name_and_id,
    _update_dataframe_datatypes,
    _base_api,
    _create_dataframe,
)

## Step 5: Create a Simple Power BI Report

Generate a basic Power BI report connected to the semantic model. This provides a visual way to confirm when data changes are reflected after framing events.

---

*Erstellen Sie einen einfachen Power BI-Bericht, der mit dem Semantic Model verbunden ist. Dies bietet eine visuelle Moeglichkeit zu bestaetigen, wann Datenaenderungen nach Framing-Ereignissen widergespiegelt werden.*

In [ ]:
report_name="Simple Report"

pbi_report:dict = {}
pbi_report['config'] = """{
        "version": "5.37",
        "themeCollection": {},
        "activeSectionIndex": 0,
        "linguisticSchemaSyncVersion": 0,
        "objects": {
            "outspacePane": [
                {
                    "properties": {
                        "expanded": {
                            "expr": {
                                "Literal": {
                                    "Value": "false"
                                }
                            }
                        }
                    }
                }
            ]
        }
    }"""
pbi_report['layoutOptimization']=0
pbi_report['resourcePackages'] = [{'resourcePackage': {'disabled': False, 'items': [{'name': 'CY24SU10', 'path': 'BaseThemes/CY24SU10.json', 'type': 202}], 'name': 'SharedResources', 'type': 2}}]
pbi_report['sections'] = [
    {'config': '{}', 
    'displayName': 'Page 1', 
    'displayOption': 1, 
    'filters': '[]', 
    'height': 300.0, 
    'width': 600.0,
    'name': 'a4c1ed461808909ae820', 
    'visualContainers':
        [
            {'config': '''{
                        "name": "Matrix",
                        "layouts": [
                            {
                                "id": 0,
                                "position": {
                                    "x": 310,
                                    "y": 30,
                                    "z": 1000,
                                    "width": 253,
                                    "height": 202
                                }
                            }
                        ],
                        "singleVisual": {
                            "visualType": "tableEx",
                            "projections": {
                                "Values": [
                                    {
                                        "queryRef": "DimDate.Month"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.SalesAmount)"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.DiscountAmount)"
                                    }
                                ]
                            },
                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "d",
                                        "Entity": "DimDate",
                                        "Type": 0
                                    },
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "d"
                                                }
                                            },
                                            "Property": "Month"
                                        },
                                        "Name": "DimDate.Month",
                                        "NativeReferenceName": "Month"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "SalesAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.SalesAmount)",
                                        "NativeReferenceName": "Sum of SalesAmount"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "DiscountAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.DiscountAmount)",
                                        "NativeReferenceName": "Discount"
                                    }
                                ]
                            },
                            "columnProperties": {
                                "Sum(FactInternetSales.DiscountAmount)": {
                                    "displayName": "Discount"
                                }
                            },
                            "drillFilterOtherVisuals": true,
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    }''', 'filters': '[]', 'height': 202.46, 'width': 215.11, 'x': 319.67, 'y': 30.63, 'z': 1.0
                },
            {'config': 
                    '''{
                        "name":"Card",
                        "layouts":[
                            {
                            "id":0,
                            "position":{"x":10,"y":30,"z":0,"width":238,"height":201}}
                            ],
                        "singleVisual":{"visualType":"card","projections":{"Values":[{"queryRef":"FactInternetSales.Count of Sales"}]},


                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Measure": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "f"
                                                }
                                            },
                                            "Property": "Count of Sales"
                                        },
                                        "Name": "FactInternetSales.Count of Sales",
                                        "NativeReferenceName": "Count of Sales"
                                    }
                                ],
                                "OrderBy": [
                                    {
                                        "Direction": 2,
                                        "Expression": {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {
                                                        "Source": "f"
                                                    }
                                                },
                                                "Property": "Count of Sales"
                                            }
                                        }
                                    }
                                ]
                            },



                            "drillFilterOtherVisuals":true,
                            "hasDefaultSort":true,                           
                            "objects": {
                                "labels": [
                                    {
                                        "properties": {
                                            "fontSize": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "20D"
                                                    }
                                                }
                                            },
                                            "labelDisplayUnits": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "1D"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            },
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                            }
                        }''',
                 'filters': '[]', 
                 'height': 201.5, 
                 'width': 265.43, 
                 'x': 270.03, 
                 'y': 30.12, 
                 'z': 1000.0
            }
        ]
    }]

labs.report.create_report_from_reportjson(report=report_name , dataset="AdventureWorks_model" , report_json = pbi_report)
report_id = resolve_report_id(report_name)

from powerbiclient import Report
simple_report = Report(group_id=None, report_id=report_id)
simple_report.set_size(400,700)
simple_report

## Step 6: Show Lakehouse Tables

List all tables in the lakehouse to confirm the data structure before modifying data.

---

*Listen Sie alle Tabellen im Lakehouse auf, um die Datenstruktur vor der Datenaenderung zu bestaetigen.*

In [ ]:
# Display all available tables in the lakehouse for framing analysis
display(labs.lakehouse.get_lakehouse_tables(lakehouse=LakehouseName))

## Step 7: Show Relationships

Display the relationships configured in the semantic model.

---

*Zeigen Sie die im Semantic Model konfigurierten Beziehungen an.*

In [ ]:
# Display semantic model relationships to understand framing dependencies
display(labs._list_functions.list_relationships(SemanticModelName))

## Step 8: Show Delta History for DimDate

Display the Delta table transaction history for DimDate to see a log of all data modification operations.

---

*Zeigen Sie die Delta-Tabellen-Transaktionshistorie fuer DimDate an, um ein Protokoll aller Datenaenderungsoperationen zu sehen.*

In [ ]:
# Analyze DimDate table modification history to establish baseline
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="DimDate"))
simple_report

## Step 9: Show Delta History for FactInternetSales

Review the transaction history of the fact table to establish a baseline before performing data modifications.

---

*Ueberpruefen Sie die Transaktionshistorie der Faktentabelle, um eine Basislinie vor der Durchfuehrung von Datenaenderungen festzulegen.*

In [ ]:
# Examine FactInternetSales table history to understand current state
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])

## Step 10: Insert Data into FactInternetSales

Append new rows to the fact table to trigger an automatic framing event. After this step, check the Power BI report to see whether the new data appears.

---

*Fuegen Sie neue Zeilen zur Faktentabelle hinzu, um ein automatisches Framing-Ereignis auszuloesen. Pruefen Sie nach diesem Schritt den Power BI-Bericht, ob die neuen Daten erscheinen.*

In [ ]:
# Get one day of data from existing table
from pyspark.sql.functions import lit, min, max ,count
df1 = spark.read.load(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")

# Show Min, MAX and Count of rows
# df1.agg(
#     min("OrderDateKey").alias("min_OrderDateKey") ,
#     max("OrderDateKey").alias("max_OrderDateKey") ,
#     count("*").alias("count_rows")
#     ).show()


# Create a filtered dataframe to update and then append back onto the original table
df2 = df1.filter("OrderDateKey='20221204'")
df2 = df2.withColumn("OrderDateKey",lit(20050630))


df2.write.mode("append").save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])
simple_report

In [ ]:
df1 = spark.read.load(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")

# Show Min, MAX and Count of rows
df1.agg(
        min("OrderDateKey").alias("min_OrderDateKey") ,
        max("OrderDateKey").alias("max_OrderDateKey") ,
        count("*").alias("count_rows")
        ).show()


## Step 11: Load FactInternetSales into a Variable

Cache the fact table data in a DataFrame for efficient use in subsequent delete and update operations.

---

*Speichern Sie die Faktentabellendaten in einem DataFrame fuer die effiziente Verwendung in nachfolgenden Loesch- und Aktualisierungsoperationen.*

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")


## Step 12: Delete Rows from FactInternetSales

Remove a subset of rows to observe how delete operations trigger framing events and affect the model.

---

*Entfernen Sie eine Teilmenge von Zeilen, um zu beobachten, wie Loeschoperationen Framing-Ereignisse ausloesen und das Modell beeinflussen.*

In [ ]:
deltaTable.delete("OrderDateKey = '20050701'")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])
simple_report

## Step 13: Update Rows in FactInternetSales

Modify existing rows to observe how update operations trigger automatic framing.

---

*Aendern Sie bestehende Zeilen, um zu beobachten, wie Aktualisierungsoperationen automatisches Framing ausloesen.*

In [ ]:
deltaTable.update(
    condition= col("OrderDateKey")=='20220218',
    set = { "DiscountAmount":"1"}
)
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])
simple_report

## Step 14: Disable Automatic Framing

Turn off automatic framing on the semantic model. This must be done manually in the semantic model settings in the Fabric portal. You may need to refresh the model afterwards to confirm the setting change.

---

*Deaktivieren Sie das automatische Framing im Semantic Model. Dies muss manuell in den Einstellungen des Semantic Models im Fabric-Portal erfolgen. Moeglicherweise muessen Sie das Modell danach aktualisieren, um die Einstellungsaenderung zu bestaetigen.*

## Step 15: Insert More Data (Framing Disabled)

Insert additional rows while automatic framing is disabled. The model will not pick up these changes until a manual reframe is triggered.

---

*Fuegen Sie zusaetzliche Zeilen ein, waehrend das automatische Framing deaktiviert ist. Das Modell uebernimmt diese Aenderungen erst nach einem manuellen Reframing.*

In [ ]:
# Create filtered dataframe for appending new data while auto framing is disabled
df2 = df1.filter("OrderDateKey='20221204'")
df2 = df2.withColumn("OrderDateKey",lit(20050629))

# Append data to table and monitor framing behavior
df2.write.mode("append").save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])
simple_report

## Step 16: Update Rows (Framing Disabled)

Perform updates with auto framing disabled. Changes accumulate in the Delta table without being reflected in the model.

---

*Fuehren Sie Aktualisierungen bei deaktiviertem Auto-Framing durch. Aenderungen sammeln sich in der Delta-Tabelle an, ohne im Modell widergespiegelt zu werden.*

In [ ]:
# Update specific rows while auto framing is disabled
deltaTable.update(
    condition= col("OrderDateKey")=='20220218',
    set = { "DiscountAmount":"2"}
)
time.sleep(4)
# Check history to see accumulated changes without automatic framing
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])
simple_report

## Step 17: Delete Rows (Framing Disabled)

Remove rows while framing is still disabled to add to the accumulated unsynced changes.

---

*Entfernen Sie Zeilen, waehrend Framing noch deaktiviert ist, um zu den angesammelten nicht synchronisierten Aenderungen beizutragen.*

In [ ]:
deltaTable.delete("OrderDateKey = '20050702'")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales")[
        ['Change Number','Change Type','Modification Time','Incremental Effect',
        'Rows After Change','Rows Added','Rows Removed','Rows Delta' ,
        'Files Added','Files Removed','Files After Change','Files Preserved' ,
        'Size Added','Sized Removed','Size After Change'
        ]
     ])
simple_report

## Step 18: Manually Reframe the Model

Trigger a manual reframe to synchronise the model with all accumulated data changes. After this step, the Power BI report should reflect all inserts, updates, and deletes performed while framing was disabled.

---

*Loesen Sie ein manuelles Reframing aus, um das Modell mit allen angesammelten Datenaenderungen zu synchronisieren. Nach diesem Schritt sollte der Power BI-Bericht alle Einfuegungen, Aktualisierungen und Loeschungen widerspiegeln.*

In [ ]:
# Manually trigger reframe operation to synchronize all accumulated changes
labs.refresh_semantic_model(SemanticModelName)
simple_report

## Step 19: Stop the Spark Session

---

*Spark-Sitzung beenden.*

In [ ]:
# Clean up resources by stopping the Spark session
mssparkutils.session.stop()

## Lab Summary

In this lab you explored how Direct Lake framing controls when a semantic model picks up new data from the underlying Delta tables. You observed automatic framing behaviour, disabled it for manual control, performed batch data operations, and reframed the model on demand.

**Key Takeaways**
- Automatic framing keeps reports in sync with lakehouse data in near-real time, but each reframe carries a small performance cost.
- Disabling automatic framing is useful during batch ETL operations where you do not want the model refreshing mid-load.
- Manual reframing gives you precise control over when the model reflects new data.
- Framing history lets you audit how often and when the model was synchronised.
- Combining batch inserts with a single manual reframe at the end is more efficient than allowing continuous automatic reframes during large writes.

**Next Lab:** Lab 6 covers column partitioning and its effect on Direct Lake query performance.

---

*In diesem Lab haben Sie untersucht, wie Direct Lake Framing steuert, wann ein semantisches Modell neue Daten aus den zugrunde liegenden Delta-Tabellen uebernimmt. Sie haben das automatische Framing-Verhalten beobachtet, es fuer manuelle Steuerung deaktiviert, Batch-Datenoperationen durchgefuehrt und das Modell bei Bedarf manuell neu gerahmt.*

**Wichtige Erkenntnisse**
- Automatisches Framing haelt Berichte nahezu in Echtzeit mit den Lakehouse-Daten synchron, wobei jeder Reframe einen kleinen Leistungsaufwand verursacht.
- Das Deaktivieren des automatischen Framings ist waehrend Batch-ETL-Operationen sinnvoll, wenn das Modell sich nicht mitten im Ladevorgang aktualisieren soll.
- Manuelles Reframing gibt Ihnen praezise Kontrolle darueber, wann das Modell neue Daten widerspiegelt.
- Der Framing-Verlauf ermoeglicht es, zu ueberpruefen, wie oft und wann das Modell synchronisiert wurde.
- Die Kombination von Batch-Einfuegungen mit einem einzigen manuellen Reframe am Ende ist effizienter als kontinuierliche automatische Reframes waehrend grosser Schreibvorgaenge.

**Naechstes Lab:** Lab 6 behandelt Column Partitioning und dessen Auswirkungen auf die Direct Lake Abfrageleistung.